In [ ]:
from nbdev import *
%nbdev_default_export hierarchy

Cells will be exported to pct.hierarchy,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%nbdev_hide
%reload_ext autoreload
%autoreload 2

In [ ]:
# hide
import sys
sys.path.append("..")

# Hierarchy

Creation of Perceptual Control hierarchies.

In [ ]:
%nbdev_export
import numpy as np
from pct.nodes import PCTNode
from pct.functions import *

In [ ]:
%nbdev_export
class PCTHierarchy():
    "A hierarchical perceptual control system, of PCTNodes."
    def __init__(self, rows=0, cols=0, pre=[], post=[], name="pcthierarchy", clear_names=True, links="single", history=False, **pargs):
        self.links_built = False
        if clear_names:
            UniqueNamer.getInstance().clear()
        self.name=UniqueNamer.getInstance().get_name(name)
        self.preCollection=pre
        self.postCollection=post
        self.hierarchy = []
        for r in range(rows):
            col_list=[]
            for c in range(cols):
                if links == "dense":
                    if r > 0:
                        perc = WeightedSum(weights=np.ones(cols))
                    if r < rows-1:
                        ref = WeightedSum(weights=np.ones(cols))
                    if r == 0:
                        node = PCTNode(reference=ref, name=f'row{r}col{c}', history=history)                        
                    if r > 0 and r == rows-1:                        
                        node = PCTNode(perception=perc, name=f'row{r}col{c}', history=history)
                    if r > 0 and r < rows-1:
                        node = PCTNode(perception=perc, reference=ref, history=history, name=f'row{r}col{c}')

                else:
                    node = PCTNode(name=f'row{r}col{c}', history=history)
                    node.build_links()
                    
                self.handle_perception_links(node, r, c, links)
                self.handle_reference_links(node, r, c, links)
                col_list.append(node)
                
            self.hierarchy.append(col_list)
    
    
    def __call__(self, verbose=False):

        for func in self.preCollection:
            if verbose:
                print(func.get_name(), end =" ")
            func(verbose)          

        if verbose:
            print()

        for row in range(len(self.hierarchy)):
            for col in range(len(self.hierarchy[row])):
                node  = self.hierarchy[row][col]
                if verbose:
                    print(node.get_name(), end =" ")
                node(verbose)
            
        for func in self.postCollection:
            if verbose:
                print(func.get_name(), end =" ")
            func(verbose)          

        if verbose:
            print()
        
        output = self.postCollection[-1].get_value()
        
        if verbose:
            print()
        
        return output
            
    def run(self, steps=None, verbose=False):
        for i in range(steps):
            out = self(verbose)
        return out
    
    def get_node(self, row, col):
        return self.hierarchy[row][col]
    
    def handle_perception_links(self, node, row, col, links_type):
        if row == 0 or links_type == None:
            return
        
        if links_type == "single":
            node.add_link("perception", self.hierarchy[row-1][col].get_function("perception"))
        
        if links_type == "dense":
            for column in range(len(self.hierarchy[row-1])):
                node.add_link("perception", self.hierarchy[row-1][column].get_function("perception"))

    def handle_reference_links(self, thisnode, row, col, links_type):
        if row == 0 or links_type == None:
            return
        
        if links_type == "single":
            thatnode = self.hierarchy[row-1][col]
            thatnode.add_link("reference", thisnode.get_function("output"))
        
        if links_type == "dense":
            for column in range(len(self.hierarchy[row-1])):
                thatnode = self.hierarchy[row-1][column]
                thatnode.add_link("reference", thisnode.get_function("output"))

                
    def get_config(self):
        config = {"type": type(self).__name__,
                    "name": self.name}        
        
        pre = {}
        for i in range(len(self.preCollection)):
            pre[f'pre{i}']=self.preCollection[0].get_config()
        config['pre']=pre

        
        levels = {}
        for row in range(len(self.hierarchy)):
            level ={'level':row}
            columns={}
            for col in range(len(self.hierarchy[row])):
                column={'col':col}
                nodeconfig = self.hierarchy[row][col].get_config()
                #print(nodeconfig)
                column['node']=nodeconfig
                #print(column)
                columns[f'col{col}']=column
            level['nodes']=columns
            levels[f'level{row}']=level
        config['levels']=levels
        
        post = {}
        for i in range(len(self.postCollection)):
            post[f'post{i}']=self.postCollection[0].get_config()
        config['post']=post
        return config       

    
    @classmethod
    def from_config(cls, config):
        #lookup
        hpct = PCTHierarchy(name=config['name'])
        preCollection = []        
        coll_dict = config['pre']
        PCTNode.collection_from_config(preCollection, coll_dict)
        
        postCollection = []        
        coll_dict = config['post']
        PCTNode.collection_from_config(postCollection, coll_dict)
     
        hpct.pre=preCollection
        hpct.post=postCollection
                
        hpct.hierarchy=[]
        for level_key in config['levels'].keys():
            cols = []
            for nodes_key in config['levels'][level_key]['nodes'].keys():
                print("PCTHierarchy from_config", nodes_key)
                print("PCTHierarchy from_config", config['levels'][level_key]['nodes'][nodes_key]['node'])
                node = PCTNode.from_config(config['levels'][level_key]['nodes'][nodes_key]['node'])
                print("PCTHierarchy from_config", node.get_config())
                cols.append(node)
            hpct.hierarchy.append(cols)
        
        
        """
        node.referenceCollection = []        
        collection = node.referenceCollection
        coll_dict = config['refcoll']
        PCTNode.collection_from_config(collection, coll_dict)

        node.perceptionCollection = []
        collection = node.perceptionCollection
        coll_dict = config['percoll']
        PCTNode.collection_from_config(collection, coll_dict)
        
        node.comparatorCollection = []
        collection = node.comparatorCollection
        coll_dict = config['comcoll']
        PCTNode.collection_from_config(collection, coll_dict)

        node.outputCollection = []
        collection = node.outputCollection
        coll_dict = config['outcoll']
        PCTNode.collection_from_config(collection, coll_dict)
        
        node.build_links()
        """
        return hpct
    
    

In [ ]:
UniqueNamer.getInstance().clear()
pre=Constant(5, name='precon')
post=Constant(10, name='postcon')
hpct = PCTHierarchy(3,3, pre=[pre], post=[post], history=True, clear_names=False, links="dense")
hpct.hierarchy


size 0 precon False precon
size 1 postcon False postcon
size 3 weighted_sum False weighted_sum
size 5 variable False variable
size 6 subtract False subtract
size 7 proportional False proportional
size 8 weighted_sum True weighted_sum1
size 9 variable True variable1
size 9 subtract True subtract1
size 9 proportional True proportional1
size 9 weighted_sum True weighted_sum2
size 10 variable True variable2
size 10 subtract True subtract2
size 10 proportional True proportional2
size 10 weighted_sum True weighted_sum3
size 10 weighted_sum True weighted_sum4
size 11 subtract True subtract3
size 11 proportional True proportional3
size 11 weighted_sum True weighted_sum5
size 11 weighted_sum True weighted_sum6
size 12 subtract True subtract4
size 12 proportional True proportional4
size 12 weighted_sum True weighted_sum7
size 12 weighted_sum True weighted_sum8
size 13 subtract True subtract5
size 13 proportional True proportional5
size 13 weighted_sum True weighted_sum9
size 14 constant False co

[[<pct.nodes.PCTNode at 0x28ea7f9ba48>,

In [ ]:
hpct.get_config()

{'type': 'PCTHierarchy',
 'name': 'pcthierarchy',
 'pre': {'pre0': {'type': 'Constant',
   'name': 'precon',
   'value': 5,
   'links': {}}},
 'levels': {'level0': {'level': 0,
   'nodes': {'col0': {'col': 0,
     'node': {'type': 'PCTNode',
      'name': 'row0col0',
      'refcoll': {'0': {'type': 'WeightedSum',
        'name': 'weighted_sum',
        'value': 0,
        'links': {0: 'proportional3', 1: 'proportional4', 2: 'proportional5'},
        'weights': array([1., 1., 1.])}},
      'percoll': {'0': {'type': 'Variable',
        'name': 'variable',
        'value': 0,
        'links': {}}},
      'comcoll': {'0': {'type': 'Subtract',
        'name': 'subtract',
        'value': 0,
        'links': {}}},
      'outcoll': {'0': {'type': 'Proportional',
        'name': 'proportional',
        'value': 0,
        'links': {},
        'gain': 10}}}},
    'col1': {'col': 1,
     'node': {'type': 'PCTNode',
      'name': 'row0col1',
      'refcoll': {'0': {'type': 'WeightedSum',
        

In [ ]:
hpct(verbose=True)

precon 5.000 
row0col0 0.000 0.000 0.000 0.000 
row0col1 0.000 0.000 0.000 0.000 
row0col2 0.000 0.000 0.000 0.000 
row1col0 0.000 0.000 0.000 0.000 
row1col1 0.000 0.000 0.000 0.000 
row1col2 0.000 0.000 0.000 0.000 
row2col0 1.000 0.000 1.000 10.000 
row2col1 1.000 0.000 1.000 10.000 
row2col2 1.000 0.000 1.000 10.000 
postcon 10.000 



10

In [ ]:
hpct1 = PCTHierarchy(3,3, pre=[pre], post=[post], history=True, links="dense")
hpct1.run(10)

size 1 weighted_sum False weighted_sum
size 3 variable False variable
size 4 subtract False subtract
size 5 proportional False proportional
size 6 weighted_sum True weighted_sum1
size 7 variable True variable1
size 7 subtract True subtract1
size 7 proportional True proportional1
size 7 weighted_sum True weighted_sum2
size 8 variable True variable2
size 8 subtract True subtract2
size 8 proportional True proportional2
size 8 weighted_sum True weighted_sum3
size 8 weighted_sum True weighted_sum4
size 9 subtract True subtract3
size 9 proportional True proportional3
size 9 weighted_sum True weighted_sum5
size 9 weighted_sum True weighted_sum6
size 10 subtract True subtract4
size 10 proportional True proportional4
size 10 weighted_sum True weighted_sum7
size 10 weighted_sum True weighted_sum8
size 11 subtract True subtract5
size 11 proportional True proportional5
size 11 weighted_sum True weighted_sum9
size 12 constant False constant
size 13 subtract True subtract6
size 13 proportional True 

10

In [ ]:
print(hpct1.get_node(1,1).history.data)
assert hpct1.get_node(1,1).history.data == {'refcoll': {'weighted_sum6': [0.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0]}, 'percoll': {'weighted_sum5': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, 'comcoll': {'subtract4': [0.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0]}, 'outcoll': {'proportional4': [0.0, 300.0, 300.0, 300.0, 300.0, 300.0, 300.0, 300.0, 300.0, 300.0]}}

{'refcoll': {'weighted_sum6': [0.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0]}, 'percoll': {'weighted_sum5': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, 'comcoll': {'subtract4': [0.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0]}, 'outcoll': {'proportional4': [0.0, 300.0, 300.0, 300.0, 300.0, 300.0, 300.0, 300.0, 300.0, 300.0]}}


In [ ]:
h = PCTHierarchy.from_config(hpct.get_config())
h.get_config()

collection_from_config {'pre0': {'type': 'Constant', 'name': 'precon', 'value': 5, 'links': {}}}
fndict_label pre0
{'name': 'precon', 'value': 5, 'links': {}}
a: {'name': 'precon', 'value': 5, 'links': {}}
size 1 precon False precon
b: {'type': 'Constant', 'name': 'precon', 'value': 5, 'links': {}}
collection_from_config {'post0': {'type': 'Constant', 'name': 'postcon', 'value': 10, 'links': {}}}
fndict_label post0
{'name': 'postcon', 'value': 10, 'links': {}}
a: {'name': 'postcon', 'value': 10, 'links': {}}
size 2 postcon False postcon
b: {'type': 'Constant', 'name': 'postcon', 'value': 10, 'links': {}}
PCTHierarchy from_config col0
PCTHierarchy from_config {'type': 'PCTNode', 'name': 'row0col0', 'refcoll': {'0': {'type': 'WeightedSum', 'name': 'weighted_sum', 'value': 0.0, 'links': {0: 'proportional3', 1: 'proportional4', 2: 'proportional5'}, 'weights': array([1., 1., 1.])}}, 'percoll': {'0': {'type': 'Variable', 'name': 'variable', 'value': 0, 'links': {}}}, 'comcoll': {'0': {'type'

{'type': 'PCTHierarchy',
 'name': 'pcthierarchy',
 'pre': {},
 'levels': {'level0': {'level': 0,
   'nodes': {'col0': {'col': 0,
     'node': {'type': 'PCTNode',
      'name': 'row0col0',
      'refcoll': {'0': {'type': 'WeightedSum',
        'name': 'weighted_sum',
        'value': 0.0,
        'links': {0: 'proportional3', 1: 'proportional4', 2: 'proportional5'},
        'weights': array([1., 1., 1.])}},
      'percoll': {'0': {'type': 'Variable',
        'name': 'variable1',
        'value': 0,
        'links': {}}},
      'comcoll': {'0': {'type': 'Subtract',
        'name': 'subtract1',
        'value': 0.0,
        'links': {0: 'weighted_sum',
         1: 'variable',
         2: 'weighted_sum',
         3: 'variable1'}}},
      'outcoll': {'0': {'type': 'Proportional',
        'name': 'proportional1',
        'value': 0.0,
        'links': {0: 'subtract', 1: 'subtract1'},
        'gain': 10}}}},
    'col1': {'col': 1,
     'node': {'type': 'PCTNode',
      'name': 'row0col1',
   

In [ ]:
assert h.get_config() == hpct.get_config()

AssertionError: 

In [ ]:
#hide
from nbdev import *
notebook2script()